# Variable dependiente: Monto Corrupción Amplia (numérica)

## 1. Data

#### 1.1. Import libraries and data

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import pandas as  pd, numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [4]:
import funciones as fun
import variables_nombres as vn

In [5]:
path = r'..\..\..\input\preprocessed_data\base1.csv'
data = pd.read_csv( path )

In [6]:
# Borrar otras variables dependientes. Solo nos quedamos con 'monto_corrup2'
dep_var    = [ 'monto_corrup2' ]
other_deps = [ element for element in vn.dependientes_variables if element not in dep_var ]

data       = data.drop( other_deps, axis = 1 )
data       = data.drop( [ 'year' ], axis = 1 )

#### 1.2. Split data into training and test set

In [7]:
np.random.seed( 1234 )
training   = np.random.choice( data.index, size=int( len( data )*( 3/4 )), replace = False )

data_train = data.loc[ training, : ]
data_test  = data.drop( training, axis = 0 )

## 2. Definir el modelo

In [8]:
import statsmodels.formula.api as smf
import patsy

#### 2.1. Construir modelos

In [9]:
base1_vars_list = [ var for var in data.columns if var not in dep_var ]
predictors      = ' + '.join( base1_vars_list )
base1_vars_df   = pd.DataFrame( base1_vars_list )
base1_vars_df.to_excel( r'..\..\..\extra\varlists\b1.xlsx' )

In [ ]:
%%time

formula_basic = f'monto_corrup2 ~ { predictors }'
y_basic_train, x_basic_train = patsy.dmatrices( formula_basic, data_train, return_type = 'dataframe' )
y_basic_test, x_basic_test = patsy.dmatrices( formula_basic, data_test, return_type = 'dataframe' )
p_basic = x_basic_train.shape[ 1 ]

#### 2.2. Generar variables dependientes

In [25]:
Y_train = data_train[ 'monto_corrup2' ]
Y_test = data_test[ 'monto_corrup2' ]

In [26]:
p_basic

1054

## 3. OLS

In [27]:
x_basic_train.head()

,Intercept,tejgfun_ct05pgercon,dfgpimpiafun_ct05pgercon,devppimfun_ct05pgercon,dfgdevpiagfun_ct05opseg,devppimfun_ct05trab,tejgfun_ct05come,dfgpimpiafun_ct05come,dfgdevpiagfun_ct05turi,piagfun_ct05agro,...,tdvgtotfun_f5trab,devppimtotfun_f5trab,tejgtotfun_f5trans,devppimtotfun_f5trans,devppimtotfun_f5turi,piagtotfun_f5viv,tejgtotfun_f5viv,tdvgtotfun_f5viv,dfgpimpiatotfun_f5viv,devppimtotfun_f5viv
339,1.0,15.386930,1.351333,4.289474,0.307406,0.0,0.0,0.0,0.00,0.000000,...,0.0,0.0,14.711309,4.399793,0.000000,0.000000,12.027408,12.027408,0.197000,4.453322
549,1.0,13.155292,0.146427,4.511438,0.000000,0.0,0.0,0.0,0.03,0.000000,...,0.0,0.0,10.787709,3.442439,4.615121,0.000000,0.000000,0.000000,0.000000,0.000000
255,1.0,0.000000,0.188744,4.526656,0.000000,0.0,0.0,0.0,0.00,0.000000,...,0.0,0.0,0.000000,0.000000,4.467280,12.611541,0.000000,11.725138,-0.050000,3.921102
83,1.0,0.000000,2.125175,4.291729,-0.171162,0.0,0.0,0.0,0.00,9.179984,...,0.0,0.0,0.000000,4.256066,0.000000,0.000000,0.000000,0.000000,0.022200,0.000000
673,1.0,0.000000,-1.846324,4.425671,-0.185236,0.0,0.0,0.0,0.00,0.000000,...,0.0,0.0,0.000000,4.483049,0.000000,0.000000,0.000000,13.844683,1.095282,4.553912


#### 3.1. Ajustar el modelo

In [28]:
fit_lm_basic = smf.ols( formula_basic, data = data_train ).fit()

#### 3.2. Calcular out-of-sample MSE y Standard Error

In [29]:
# Only MSE

yhat_lm_basic = fit_lm_basic.predict( data_test )
print("The mean squared error (MSE) using the basic model is equal to:", ( ( Y_test - yhat_lm_basic )**2 ).mean() )

The mean squared error (MSE) using the basic model is equal to: 113.16806554148081


In [30]:
# MSE and SE

resid_basic  = ( Y_test - yhat_lm_basic )**2

MSE_lm_basic = sm.OLS( resid_basic , np.ones( resid_basic.shape[ 0 ] ) ).fit().summary2().tables[ 1 ].iloc[ 0, 0:2 ]
MSE_lm_basic

Coef.       113.168066
Std.Err.     15.907454
Name: const, dtype: float64

#### 3.3. Calcular R Cuadrado out-of-sample

In [31]:
R2_lm_basic = 1 - ( MSE_lm_basic[ 0 ]/Y_test.var() )
print( f"The R^2 using the basic model is equal to: { R2_lm_basic }" )

The R^2 using the basic model is equal to: -2.794676516985482


## 4. Lasso, Ridge and Elastic Net

#### 4.1. Theoretical Lasso from hdm package

In [32]:
import hdmpy as hdm

In [36]:
fit_rlasso = hdm.rlasso( x_basic_train.to_numpy() , Y_train.to_numpy().reshape( Y_train.size , 1 ) , post = False )
fit_rlasso_post = hdm.rlasso( x_basic_train.to_numpy() , Y_train.to_numpy().reshape( Y_train.size , 1 ) , post = True )

In [39]:
# Getting mean of each variable
meanx = x_basic_test.mean( axis = 0 ).values.\
                        reshape( x_basic_test.shape[ 1 ] , 1 )

# Reducing the mean
new_x1 = x_basic_test.to_numpy() - \
                    (np.ones( ( x_basic_test.shape[ 0 ] , 1 ) ) @ meanx.T)

# Getting the significant variables
x1_est_rlasso = new_x1[ :, fit_rlasso.est['index'].iloc[:, 0].to_list()]

# Getting the coef. from significant variables
beta_rlasso = fit_rlasso.est['beta'].loc[ fit_rlasso.est['index'].\
                                     iloc[:, 0].to_list(), ].to_numpy()

# yhat
yhat_rlasso = (x1_est_rlasso @ beta_rlasso) + np.mean( Y_test.to_numpy() )
residuals_rlasso = Y_test.to_numpy().reshape( Y_test.to_numpy().size, 1)  - yhat_rlasso

In [40]:
# Getting mean of each variable
meanx = x_basic_test.mean( axis = 0 ).values.\
                        reshape(x_basic_test.shape[ 1 ] , 1 )

# Reducing the mean
new_x1 = x_basic_test.to_numpy() - \
                    (np.ones( (x_basic_test.shape[ 0 ] , 1 ) ) @ meanx.T)

# Getting the significant variables
x1_est_rlasso_post = new_x1[ :, fit_rlasso_post.est['index'].iloc[:, 0].to_list()]

# Getting the coef. from significant variables
beta_rlasso_post = fit_rlasso_post.est['beta'].loc[ fit_rlasso_post.est['index'].\
                                     iloc[:, 0].to_list(), ].to_numpy()

# yhat
yhat_rlasso_post = (x1_est_rlasso_post @ beta_rlasso_post) + np.mean( Y_test.to_numpy() )
residuals_rlasso_post = Y_test.to_numpy().reshape( Y_test.to_numpy().size, 1)  - yhat_rlasso_post

In [41]:
MSE_lasso = sm.OLS( ( residuals_rlasso )**2 , np.ones( yhat_rlasso.size )  ).fit().summary2().tables[1].round(3)
MSE_lasso_post = sm.OLS( ( residuals_rlasso_post )**2  , np.ones( yhat_rlasso_post.size )  ).fit().summary2().tables[1].round(3)

R2_lasso = 1 - MSE_lasso.iloc[0, 0]/ np.var( Y_test )
R2_lasso_post = 1 - MSE_lasso_post.iloc[0, 0]/ np.var( Y_test )

print( f"The R^2 using the basic model is equal to {R2_lasso},for lasso and {R2_lasso_post} for post-lasso")

The R^2 using the basic model is equal to 0.04355234275579334,for lasso and -0.015532185134332188 for post-lasso


#### 4.2. Cross Validated Lasso, Ridge and Elastic Net

In [42]:
from sklearn.linear_model import LassoCV, RidgeCV, ElasticNetCV

In [43]:
fit_lasso_cv = LassoCV( cv = 10, fit_intercept = True, normalize = False, random_state = 0 ).fit( x_basic_train, Y_train )
fit_ridge = RidgeCV( cv = 10, fit_intercept = True, normalize = False, scoring = None ).fit( x_basic_train, Y_train )
fit_elnet = ElasticNetCV( cv = 10, fit_intercept = True, normalize = False, random_state = 0 ).fit( x_basic_train, Y_train )

yhat_lasso_cv = fit_lasso_cv.predict( x_basic_test )
yhat_ridge = fit_ridge.predict( x_basic_test )
yhat_elnet = fit_elnet.predict( x_basic_test )

residual_lasso = ( yhat_lasso_cv - Y_test )**2
residual_ridge = ( yhat_ridge - Y_test )**2
residual_elnet = ( yhat_elnet - Y_test )**2

MSE_lasso_cv = sm.OLS( residual_lasso, np.ones( Y_test.size )).fit().summary2().tables[ 1 ].round( 3 )
MSE_ridge = sm.OLS( residual_ridge, np.ones( Y_test.size )).fit().summary2().tables[ 1 ].round( 3 )
MSE_elnet = sm.OLS( residual_elnet, np.ones( Y_test.size )).fit().summary2().tables[ 1 ].round( 3 )

R2_lasso_cv = 1 - MSE_lasso_cv.iloc[ 0, 0 ] / np.var( Y_test )
R2_ridge = 1 - MSE_ridge.iloc[ 0, 0 ]  / np.var( Y_test )
R2_elnet = 1 - MSE_elnet.iloc[ 0, 0 ]  / np.var( Y_test )

print( "R^2 using cross-validation for lasso, ridge, and elastic net in the basic model: {:.5f}, {:.5f}, {:.5f}".format( R2_lasso_cv, R2_ridge, R2_elnet ) )

R^2 using cross-validation for lasso, ridge, and elastic net in the basic model: 0.02706, -2.19738, 0.02622


## 5. Non Linear Models

#### 5.1. Regression Trees

In [44]:
from sklearn.tree import DecisionTreeRegressor
from sklearn import tree

In [45]:
# Fit the model

fit_trees = DecisionTreeRegressor( random_state = 0, min_impurity_decrease = 0.001 )
fit_trees.fit( x_basic_train, y_basic_train )

DecisionTreeRegressor(min_impurity_decrease=0.001, random_state=0)

In [ ]:
# Plot the tree

# from sklearn.tree import plot_tree

# plt.figure( figsize=( 30, 20 ) )
# plot_tree( fit_trees, filled = True )
# plt.show()

In [46]:
# Determine the optimar complexity of the regression tree

s = pd.DataFrame( fit_trees.cost_complexity_pruning_path( y_basic_train, x_basic_train ) )
s.head()

,ccp_alphas,impurities
0,0.000000,1.467051
1,0.004601,1.471652
2,0.005304,1.476956
3,0.005389,1.482345
4,0.005523,1.487868


In [47]:
# Prune the tree

fit_prunnedtree = DecisionTreeRegressor( ccp_alpha = 0.00188444410871555 )
fit_prunnedtree.fit( x_basic_train, y_basic_train )

DecisionTreeRegressor(ccp_alpha=0.00188444410871555)

In [ ]:
# Plot the prunned tree

# plot_tree(fit_prunnedtree, filled=True)
# plt.show()

In [48]:
# Calculate MSE and R2 for prunned tree

y_hat_pt = fit_prunnedtree.predict( x_basic_test )
residual_pt = ( y_hat_pt - Y_test )**2
MSE_pt = sm.OLS( residual_pt, np.ones( y_hat_pt.size )).fit().summary2().tables[ 1 ].round( 3 )
R2_pt = 1 - MSE_pt.iloc[ 0, 0 ]/np.var( Y_test )
print( f"R^2 of the pruned tree: { R2_pt }" )

R^2 of the pruned tree: -0.8148932379765956


#### 5.2. Random Forest and Booested Trees

In [49]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor 

In [50]:
# random forest
fit_rf = RandomForestRegressor( n_estimators = 2000, min_samples_leaf = 5).fit( x_basic_train, Y_train )

# boosting
fit_boost = GradientBoostingRegressor( loss = 'ls', learning_rate = 0.01, n_estimators = 1000, max_depth = 2, subsample = 0.5).fit( x_basic_train, Y_train )

# Evaluating the methods
yhat_rf = fit_rf.predict( x_basic_test )
residual_rf = ( yhat_rf - Y_test )**2
yhat_boost = fit_boost.predict( x_basic_test )
residual_bst = ( yhat_boost - Y_test )**2

# Calculate MSE
MSE_rf = sm.OLS( residual_pt, np.ones( yhat_rf.size )).fit().summary2().tables[ 1 ].round( 3 )
MSE_bst = sm.OLS( residual_bst, np.ones( yhat_boost.size )).fit().summary2().tables[ 1 ].round( 3 )

# Calculate R2
R2_rf = 1 - MSE_rf.iloc[ 0, 0 ] / Y_test.var()
R2_boost = 1 - MSE_bst.iloc[ 0, 0 ] / Y_test.var()

In [51]:
print("R^2 of the random forest and boosted trees:{:.5f}, {:.5f}".format( R2_rf, R2_boost ))

R^2 of the random forest and boosted trees:-0.80452, 0.03554


## 6. Resultados

In [52]:
table = pd.DataFrame(columns=["MSE", "S.E for MSE", "R-squared"]) 
table.loc[0]  = [MSE_lm_basic[0], MSE_lm_basic[1], R2_lm_basic]
table.loc[1]  = [MSE_lasso.iloc[0, 0], MSE_lasso.iloc[0, 1], R2_lasso]
table.loc[2]  = [MSE_lasso.iloc[0, 0], MSE_lasso_post.iloc[0, 1], R2_lasso_post]
table.loc[3]  = [MSE_lasso_cv.iloc[0, 0], MSE_lasso_cv.iloc[0, 1], R2_lasso_cv]
table.loc[4]  = [MSE_ridge.iloc[0, 0], MSE_ridge.iloc[0, 1], R2_ridge]
table.loc[5]  = [MSE_elnet.iloc[0, 0], MSE_elnet.iloc[0, 1], R2_elnet]
table.loc[6]  = [MSE_rf.iloc[0, 0], MSE_rf.iloc[0, 1], R2_rf]
table.loc[7]  = [MSE_bst.iloc[0, 0], MSE_bst.iloc[0, 1], R2_boost]
table.loc[8]  = [MSE_pt.iloc[0, 0], MSE_pt.iloc[0, 1], R2_pt]
models_row    = [ "Least Squares (basic)", "Lasso", "Post_Lasso", "Cross-Validated lasso", 
                  "Cross-Validated ridge", "Cross-Validated elnet", "Random Forest", 
                  "Boosted Trees", "Pruned Tree" ]
table.insert( 0, "Models", models_row )
table

,Models,MSE,S.E for MSE,R-squared
0,Least Squares (basic),113.168066,15.907454,-2.794677
1,Lasso,28.361000,3.373000,0.043552
2,Post_Lasso,28.361000,3.582000,-0.015532
3,Cross-Validated lasso,28.850000,3.582000,0.027061
4,Cross-Validated ridge,94.810000,13.006000,-2.197377
5,Cross-Validated elnet,28.875000,3.587000,0.026218
6,Random Forest,53.816000,6.490000,-0.804522
7,Boosted Trees,28.763000,3.532000,0.035538
8,Pruned Tree,53.816000,6.490000,-0.814893


In [ ]:
# Tabla general para Test
table.to_excel( r'..\..\..\output\ejecucion_1\results\base1_mca_results_training.xlsx' )